In [1]:
import os, sys, re, gc, glob, numba
import torch
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from utils import logger, assign_labels, build_children_features
from IPython.display import display, clear_output, HTML
import datetime as dt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack
import pickle
from collections import defaultdict

from tqdm.auto import trange
from collections import Counter

START_DT = dt.datetime.now()


2021-03-29 23:29:17,538 - INFO - dataset:dataset.py:27 - dataset package is loaded...


In [2]:
@numba.jit(forceobj=True)
def get_parents_list(tree_dict:dict, element_id:str, paternts_list:list=None) -> list:
    """
        returns ordered list of parent for a element
        starting from root which is the <html/> tag
    """
    if paternts_list is None:
        paternts_list = []
    
    parent_id = tree_dict.get(element_id)
    if parent_id is None:
        return paternts_list
    else:
        paternts_list.append(parent_id)
        return get_parents_list(tree_dict, element_id=parent_id, paternts_list=paternts_list)


In [3]:
def build_tree_dict(df:pd.DataFrame) -> dict:
    """
        Builds tree dict for
        get_parents_list 
    """
    tree_dict = dict(zip(df.element_id.values, df.parent_id.values))
    return tree_dict


In [4]:
def build_tree_features(elements_df:pd.DataFrame) -> pd.DataFrame:
    
    def empty_string():
        return ''

    tree_dict = build_tree_dict(elements_df)
    tag_name_dict = dict(zip(elements_df.element_id.values, elements_df.tag_name.values))
    width_dict = dict(zip(elements_df.element_id.values, elements_df.width.values))
    height_dict = dict(zip(elements_df.element_id.values, elements_df.height.values))

    # Build paths
    followers_counter = Counter()
    level_dict = defaultdict(int)
    children_tags_dict = defaultdict(empty_string)
    
    with trange(elements_df.shape[0]) as tbar:
        tbar.set_description('Build tree features')
        for i, r in elements_df.iterrows(): 
            list_of_parents = get_parents_list(tree_dict=tree_dict, element_id= r.element_id)
            children_tags_dict[r.parent_id] += r.tag_name.lower()+' '
            #print(list_of_parents)
            followers_counter.update(list_of_parents)  # calculate number of followers
            tbar.update(1)
            
    elements_df['children_tags'] = elements_df.element_id.map(children_tags_dict).fillna('')
    elements_df['num_followers'] = elements_df.element_id.map(followers_counter)
    return elements_df


In [5]:
elements_df = pd.read_parquet('dataset/df/bootstrap-reboot.parquet')

In [6]:
len(build_tree_dict(elements_df))

1043

In [7]:
build_tree_features(elements_df=elements_df);

  0%|          | 0/1043 [00:00<?, ?it/s]

In [8]:
# elements_df[['parent_id', 'element_id', 'tag_name', 'children_tags', 'num_followers']].tail(30)

In [9]:
class JDIDataset(Dataset):
    
    def __init__(self, dataset_names:list=None, rebalance=False):
        super(JDIDataset, self).__init__()
        self.rebalanced = rebalance
               
        with open('dataset/classes.txt', 'r') as f:
            self.classes_dict = { class_name.strip():i for i, class_name in enumerate(f.readlines()) }
            self.classes_reverse_dict = { v:k for k, v in self.classes_dict.items()}
        self.dummy_class_value = self.classes_dict['n/a']
            
        if dataset_names is None:
            logger.warning('Using all available data to generate dataset')
            dataset_names = self._gen_dataset_names()
            
        logger.info(f"List of dataset_names:{dataset_names}")
        
        ds_list=[] # list of datasets to join
        
        for ds_name in dataset_names:
            logger.info(f'Dataset for {ds_name}')
            df = pd.read_parquet(f'dataset/df/{ds_name}.parquet')
            logger.info(f"Dataset shape: {df.shape}")

            logger.info('cleaning tag_name from dummy/auxiliary words')
            df.tag_name = df.tag_name.apply(lambda x: x.lower().replace('-example', '')) ### tag_name LOWER()
            df = build_children_features(df=df)
            df = build_tree_features(df)

            #----------------------------------------------------------------------------------------------
            # Merge children with parents
            # WARNING: There is a tag <HTML> without parent. Let's fix this issue
            df.parent_id = df.apply(lambda r: r.element_id if r.parent_id is None else r.parent_id, axis=1)            
            df = df.merge(df, left_on='parent_id', right_on='element_id', suffixes=('', '_parent'))
            logger.info(f"Dataset shape after merging with parents: {df.shape}")
            #----------------------------------------------------------------------------------------------
            
            # If annotation file exists, lets load it and assign labels
            if os.path.exists(f'dataset/annotations/{ds_name}.txt'):
                logger.warning(f'Load LABELS from dataset/annotations/{ds_name}.txt')
                img = plt.imread(f'dataset/images/{ds_name}.png')
                df = assign_labels(df, annotations_file_path=f'dataset/annotations/{ds_name}.txt', 
                                       img=img,
                                       dummy_value = self.dummy_class_value
                                  )
                del img
            else:
                logger.warning(f'assign dummy values [n/a] for labels if there is no annotations')
                df['label'] = self.dummy_class_value
            
            df['ds_name'] = ds_name
            ds_list.append(df)
            gc.collect()
                
        logger.info('Concatenate datasets')
        self.dataset = pd.concat(ds_list)
        logger.info(f"Dataset shape after reading: {self.dataset.shape}")
        
        if rebalance:
            self._oversample()
        
        self._count_vectorizer_class()

        ### add ohe_ columns to one hot encoding several attributes
        for attr in ['role', 'type', 'ui']:
            logger.info(f'Build OHE column for attribute {attr}')
            self.dataset['ohe_' + attr] = self.dataset['attributes'].apply(lambda x: x.get(attr)).fillna("").str.lower()
            
        for attr in ['role', 'type', 'ui']:
            logger.info(f'Build OHE column for attribute {attr}_parent')
            self.dataset['ohe_' + attr+'_parent'] = self.dataset['attributes_parent'].apply(lambda x: x.get(attr)).fillna("").str.lower()
            
        logger.info('OHE tag_name')
        self.tag_name_sm = self._ohe_column('tag_name')
        logger.info(f'tag_name_sm.shape: {self.tag_name_sm.shape}')

        logger.info('OHE tag_name_parent')
        self.tag_name_parent_sm = self._ohe_column(colname='tag_name_parent', ohe_file_path='model/tag_name.pkl')
        logger.info(f'tag_name_parent_sm.shape: {self.tag_name_parent_sm.shape}')

        logger.info('OHE ohe_role')
        self.ohe_role_sm = self._ohe_column('ohe_role')
        logger.info(f'ohe_role_sm.shape: {self.ohe_role_sm.shape}')

        logger.info('OHE ohe_role_parent')
        self.ohe_role_parent_sm = self._ohe_column(colname='ohe_role_parent', ohe_file_path='model/ohe_role.pkl')
        logger.info(f'ohe_role_parent_sm.shape: {self.ohe_role_parent_sm.shape}')
        
        logger.info('OHE ohe_type')
        self.ohe_type_sm = self._ohe_column('ohe_type')
        logger.info(f'ohe_type_sm.shape: {self.ohe_type_sm.shape}')

        logger.info('OHE ohe_type_parent')
        self.ohe_type_parent_sm = self._ohe_column(colname='ohe_type_parent', ohe_file_path='model/ohe_type.pkl')
        logger.info(f'ohe_type_parent_sm.shape: {self.ohe_type_parent_sm.shape}')
        
        logger.info('OHE ohe_ui')
        self.ohe_ui_sm = self._ohe_column('ohe_ui')
        logger.info(f'ohe_ui_sm.shape: {self.ohe_ui_sm.shape}')

        logger.info('OHE ohe_ui_parent')
        self.ohe_ui_parent_sm = self._ohe_column('ohe_ui_parent', ohe_file_path='model/ohe_ui.pkl')
        logger.info(f'ohe_ui_parent_sm.shape: {self.ohe_ui_parent_sm.shape}')       
        
        ## extract all non null attributes names
        self.dataset['attributes_text'] = self.dataset.attributes.apply(lambda x: " ".join([k for k in x.keys() if x[k] is not None ]))
        logger.info('Fit CountVectorizer for column "attributes"')
        self.attributes_sm = self._count_vectorizer_column('attributes_text')
        logger.info(f'attributes_sm.shape: {self.attributes_sm.shape}')    

        ## children_tags
        file_path='model/count_vectorizer_children_tags.pkl'
        if os.path.exists(file_path):
            with open(file_path, 'rb') as f:
                logger.warning(f'Load CountVectorizer for column "chidren_tags": {file_path}')
                self.count_vectorizer_chilgren_tags = pickle.load(f)
        else:
            logger.warning(f'Saving CountVectorizer for "children_tags": {file_path}')
            self.count_vectorizer_chilgren_tags = CountVectorizer().fit(self.dataset.children_tags.values)
            with open(file_path, 'wb') as f:
                pickle.dump(self.count_vectorizer_chilgren_tags, f)
        logger.info(f'CountVectorizer "chilren_tags" size: {len(self.count_vectorizer_chilgren_tags.vocabulary_)}')
        self.children_tags_sm = self.count_vectorizer_chilgren_tags.transform(self.dataset.children_tags.values)
        logger.info(f"chidren_tags_sm: {self.children_tags_sm.shape}")
        

        ## extract all non null attributes names
        self.dataset['attributes_parent_text'] = self.dataset.attributes_parent.apply(lambda x: " ".join([k for k in x.keys() if x[k] is not None ]))
        logger.info('Fit CountVectorizer for column "attributes_parent"')
        self.attributes_parent_sm = self._count_vectorizer_column(colname='attributes_parent_text', 
                                                                  file_path='model/count_vectorizer_attributes_text.pkl')
        logger.info(f'attributes_parent_sm.shape: {self.attributes_parent_sm.shape}')
        
        self._extract_features()
        
        self.labels = self.dataset.label.astype(int).map(self.classes_reverse_dict)
        self.dataset.label = self.dataset.label.astype(int)
        
        self.data = hstack([
                    self.attributes_sm, 
                    self.tag_name_sm, 
                    self.ohe_role_sm, 
                    self.ohe_type_sm, 
                    self.ohe_ui_sm,
                    self.tag_name_parent_sm,
                    self.ohe_type_parent_sm,
                    self.ohe_role_parent_sm,
                    self.ohe_ui_parent_sm,
                    self.attributes_parent_sm,
                    self.features_df.values,
                    self.children_tags_sm,
                    self.class_sm
                  ]).astype(np.float32)
        
        logger.info(f'OHE columns sparse matrix: {self.data.shape}')
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        return np.array(self.data.getrow(idx).todense()[0]).squeeze(), self.dataset.iloc[idx]['label']
    

    def _oversample(self):
        logger.warning('Oversample data to balance the dataset, this will create duplicated rows in dataset')
        
        class_counts = [ r for r in self.dataset.label.value_counts().sort_values(ascending=False).items()]
        max_count = class_counts[0][1]

        dfs = [self.dataset]
        for cc in class_counts[1:]:
            ratio = max_count//cc[1]
            ratio = 30 if ratio >= 30 else ratio
            for _ in range(ratio):
                dfs.append(self.dataset[self.dataset.label == cc[0]].copy())

        self.dataset = pd.concat(dfs)
        logger.warning(f'Rebalanced dataset size: {self.dataset.shape[0]}')
        
                
    def _ohe_column(self, colname, ohe_file_path=None):
        """
            load attr_ohe if exists model/attr_ohe.pkl
            otherwise build the one
        """
        if ohe_file_path is None:
            file_path = f'model/{colname}.pkl'
        else:
            file_path = ohe_file_path
            
        if os.path.exists(file_path):
            logger.warning(f'loading existing OHE for column "{colname}" from {file_path}')
            with open(file_path,'rb') as f: 
                ohe = pickle.load(f)
        else:
            logger.warning(f'Create and fit OHE for column "{colname}"')
            ohe = OneHotEncoder(handle_unknown='ignore')
            ohe.fit(np.expand_dims(self.dataset[colname].values, axis=1))            
            with open(file_path, 'wb') as f:
                pickle.dump(ohe, f)
                
        sm = ohe.transform(np.expand_dims(self.dataset[colname].values, axis=1))        
        return sm
        
    def _count_vectorizer_column(self, colname, file_path=None):
        """
            load count_vercorizer for a column if a pkl file exists
            otherwise create the one
        """
        if file_path is None:
            file_path = f'model/count_vectorizer_{colname}.pkl'
            
        if os.path.exists(file_path):
            logger.warning(f'loading existing count vectorizer for column "{colname}" from {file_path}')
            with open(file_path,'rb') as f: 
                vectorizer = pickle.load(f)
            self.vocabulary = vectorizer.vocabulary_
        else:
            logger.warning(f'Create and fit count vectorizer for column "{colname}"')
            vectorizer = CountVectorizer(vocabulary=self._build_vocabulary())
            vectorizer.fit(self.dataset[colname].values)            
            with open(file_path, 'wb') as f:
                pickle.dump(vectorizer, f)
                
        sm = vectorizer.transform(self.dataset[colname].values)
        return sm
    
    def _build_vocabulary(self):
        
        """
            Attempt to reduce number of features by removing rarely used attributes
        """
        
        attributes_usage = Counter()
        for attr_list in self.dataset.attributes.apply(lambda x: [field for field in x if x[field] is not None]).values:
            attributes_usage.update(attr_list)
            
        attributes_usasge_df = pd.DataFrame( 
            [[attribute, cnt] for attribute, cnt in attributes_usage.items()], 
            columns=['attribute', 'cnt']
        ).sort_values(by='cnt', ascending=False)

        ## Lets cut off attributes which are rarely used
        attributes_list_df = attributes_usasge_df[attributes_usasge_df.cnt>1].copy()
        
        attr_unique_values_map = { 
                                    attr:self.dataset['attributes'].apply(lambda x: x.get(attr)).nunique() 
                                         for attr in (attributes_list_df.attribute.values)
                                 }
        attributes_list_df['num_unique_values'] = attributes_list_df.attribute.map(attr_unique_values_map)
        attributes_list_df['k'] = attributes_list_df.cnt/attributes_list_df.num_unique_values
        
        attributes_list_df = attributes_list_df[(attributes_list_df.k>3.0) & (attributes_list_df.num_unique_values>1)]\
                                    .sort_values(by='cnt', ascending=False).copy()
        
        self.vocabulary = { w:i for i,w in enumerate(sorted(attributes_list_df.attribute.values))}
        return self.vocabulary
        
        
    def _extract_features(self):
        self.features_df = self.dataset[[
            #'tag_name_parent',
            #'tag_name',
            'width', 
            'height', 
            'width_parent', 
            'height_parent', 
            'x', 
            'x_parent', 
            'y', 
            'y_parent',
            'is_leaf',
            'is_leaf_parent',
            'num_followers',
            'num_leafs',
            #'num_leafs_parent',
            'num_children',
            'num_children_parent',
            'sum_children_widths',
            'sum_children_widths_parent',
            'sum_children_hights',
            'sum_children_hights_parent',
            'displayed',
            #'onmouseenter'
        ]].copy()
        
        self.features_df.sum_children_hights = (self.features_df.sum_children_hights/self.features_df.num_children).fillna(-1)
        self.features_df.sum_children_hights_parent = (self.features_df.sum_children_hights_parent/self.features_df.num_children_parent).fillna(-1)
        self.features_df.sum_children_widths = (self.features_df.sum_children_widths/self.features_df.num_children).fillna(-1)
        self.features_df.sum_children_widths_parent = (self.features_df.sum_children_widths_parent/self.features_df.num_children_parent).fillna(-1)

        self.features_df.x = (self.features_df.x < 0).astype(int)
        self.features_df.y = (self.features_df.y < 0).astype(int)
        self.features_df.x_parent = (self.features_df.x_parent < 0).astype(int)
        self.features_df.y_parent = (self.features_df.y_parent < 0).astype(int)
        self.features_df['w'] = (self.features_df.width <= 2).astype(int)
        self.features_df['w_parent'] = (self.features_df.width_parent <= 2).astype(int)
        self.features_df['h'] = (self.features_df.height <= 2).astype(int)
        self.features_df['h_parent'] = (self.features_df.height_parent <= 2).astype(int)
        self.features_df.displayed = self.features_df.displayed.astype(int)
        
        
    def _find_dataset_names(self, path_mask='dataset/df/*.parquet'):
        return  set([re.sub( '.*[/\\\]', '', re.sub('\\..*$', '', os.path.normpath(fn)))
                        for fn in glob.glob(path_mask)])


    def _gen_dataset_names(self):
        dfs = self._find_dataset_names('dataset/df/*.parquet')
        imgs = self._find_dataset_names('dataset/images/*.png')
        anns = self._find_dataset_names('dataset/annotations/*.txt')

        return (dfs.intersection(imgs)).intersection(anns)
        
    def _count_vectorizer_class(self):

        self.dataset['cv_class'] = self.dataset.attributes.apply(lambda x: x.get('class')).fillna('')
        file_name = 'model/count_vectorizer_class.pkl'
        if os.path.exists(file_name):
            logger.warning(f'Loading count vectorizer for column "cv_class": {file_name}')
            with open(file_name, 'rb') as f:
                self.count_vectorizer_class = pickle.load(f)
        else:
            logger.warning(f'Build count vectorizer for column "cv_class": {file_name}')
            class_dict = Counter()
            for s in self.dataset['cv_class'].values:
                class_dict.update(s.lower().replace('-', ' ').split())
            vocabulary = [cls for cls in class_dict if re.match('^[a-z]*$', cls)]
            self.count_vectorizer_class = CountVectorizer(vocabulary=vocabulary).fit(self.dataset['cv_class'].values)
            with open(file_name, 'wb') as f:
                pickle.dump(self.count_vectorizer_class, f)
        self.class_sm = self.count_vectorizer_class.transform(self.dataset['cv_class'].values)
        logger.info(f'class_sm: {self.class_sm.shape}')

    

    

In [10]:
train_dataset = JDIDataset(rebalance=True)

2021-03-29 23:29:19,180 - WARNING - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:13 - Using all available data to generate dataset
2021-03-29 23:29:19,183 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:16 - List of dataset_names:{'user-table', 'react-ant', 'ms-office', 'different-elemants', 'bootstrap-form', 'search', 'bootstrap', 'bootstrap-forms', 'html-5', 'support', 'dates', 'contact-form', 'performance', 'metals-and-colors', 'bootstrap-form-control', 'bootstrap-1', 'bootstrap-reboot', 'complex-table', 'table-with-pages', 'angular', 'google-voice', 'gitlab', 'mobile-and-html-5', 'login'}
2021-03-29 23:29:19,184 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:21 - Dataset for user-table
2021-03-29 23:29:19,202 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:23 - Dataset shape: (369, 13)
2021-03-29 23:29:19,203 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:25 - cleaning tag

  0%|          | 0/369 [00:00<?, ?it/s]

2021-03-29 23:29:19,715 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:35 - Dataset shape after merging with parents: (369, 40)
2021-03-29 23:29:19,716 - WARNING - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:40 - Load LABELS from dataset/annotations/user-table.txt


Assigning labels:   0%|          | 0/58 [00:00<?, ?it/s]

2021-03-29 23:29:19,982 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:21 - Dataset for react-ant
2021-03-29 23:29:20,017 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:23 - Dataset shape: (703, 13)
2021-03-29 23:29:20,017 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:25 - cleaning tag_name from dummy/auxiliary words
2021-03-29 23:29:20,017 - INFO - dataset:dataset.py:59 - select all leafs (nodes which are not parents)
2021-03-29 23:29:20,032 - INFO - dataset:dataset.py:61 - Leafs set size: 261 (nodes which have no children)
2021-03-29 23:29:20,034 - INFO - dataset:dataset.py:64 - count number of references to leafs
2021-03-29 23:29:20,035 - INFO - dataset:dataset.py:66 - Nodes with leafs as children set size: 199 (nodes which have leafs as children)
2021-03-29 23:29:20,036 - INFO - dataset:dataset.py:69 - count num children for each node
2021-03-29 23:29:20,037 - INFO - dataset:dataset.py:71 - Nodes with children

...

2021-03-29 23:30:12,550 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:21 - Dataset for login
2021-03-29 23:30:12,553 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:23 - Dataset shape: (231, 13)
2021-03-29 23:30:12,553 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:25 - cleaning tag_name from dummy/auxiliary words
2021-03-29 23:30:12,553 - INFO - dataset:dataset.py:59 - select all leafs (nodes which are not parents)
2021-03-29 23:30:12,553 - INFO - dataset:dataset.py:61 - Leafs set size: 100 (nodes which have no children)
2021-03-29 23:30:12,569 - INFO - dataset:dataset.py:64 - count number of references to leafs
2021-03-29 23:30:12,571 - INFO - dataset:dataset.py:66 - Nodes with leafs as children set size: 72 (nodes which have leafs as children)
2021-03-29 23:30:12,572 - INFO - dataset:dataset.py:69 - count num children for each node
2021-03-29 23:30:12,573 - INFO - dataset:dataset.py:71 - Nodes with children: 132

  0%|          | 0/231 [00:00<?, ?it/s]

2021-03-29 23:30:12,885 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:35 - Dataset shape after merging with parents: (231, 40)
2021-03-29 23:30:12,886 - WARNING - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:40 - Load LABELS from dataset/annotations/login.txt


Assigning labels:   0%|          | 0/23 [00:00<?, ?it/s]

2021-03-29 23:30:12,985 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:55 - Concatenate datasets
2021-03-29 23:30:13,051 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:57 - Dataset shape after reading: (27008, 42)
2021-03-29 23:30:13,051 - WARNING - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:165 - Oversample data to balance the dataset, this will create duplicated rows in dataset
2021-03-29 23:30:14,029 - WARNING - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:178 - Rebalanced dataset size: 82062
2021-03-29 23:30:14,047 - WARNING - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:320 - Loading count vectorizer for column "cv_class": model/count_vectorizer_class.pkl
2021-03-29 23:30:14,329 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:333 - class_sm: (82062, 779)
2021-03-29 23:30:14,329 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:66 - Build 

In [11]:
train_dataset.dataset.columns

Index(['attributes', 'displayed', 'element_id', 'height', 'onmouseenter',
       'onmouseover', 'parent_id', 'style', 'tag_name', 'text', 'width', 'x',
       'y', 'is_leaf', 'num_leafs', 'num_children', 'sum_children_widths',
       'sum_children_hights', 'children_tags', 'num_followers',
       'attributes_parent', 'displayed_parent', 'element_id_parent',
       'height_parent', 'onmouseenter_parent', 'onmouseover_parent',
       'parent_id_parent', 'style_parent', 'tag_name_parent', 'text_parent',
       'width_parent', 'x_parent', 'y_parent', 'is_leaf_parent',
       'num_leafs_parent', 'num_children_parent', 'sum_children_widths_parent',
       'sum_children_hights_parent', 'children_tags_parent',
       'num_followers_parent', 'label', 'ds_name', 'cv_class', 'ohe_role',
       'ohe_type', 'ohe_ui', 'ohe_role_parent', 'ohe_type_parent',
       'ohe_ui_parent', 'attributes_text', 'attributes_parent_text'],
      dtype='object')

In [12]:
assert (train_dataset.dataset.element_id.nunique() == train_dataset.dataset.shape[0]) or train_dataset.rebalanced, 'There are nonunique values in the dataset, probably the dataset was rebalanced'
display(HTML("<h3>Check dataset.element_id for uniqueness: OK</h3>"))

In [13]:
display(HTML(f'<h3>Length of dataset: {train_dataset.dataset.shape[0]}<h3>'))

In [14]:
train_dataset.data.shape

(82062, 2002)

## Test Dataloader

In [15]:
BATCH_SIZE = 256
dataloader = DataLoader(train_dataset, shuffle=True, batch_size = BATCH_SIZE)

In [16]:
%%time
next(iter(dataloader))

Wall time: 5.66 s


[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([ 1, 12, 20, 20, 20, 20,  1,  0,  1,  1,  0,  1,  1,  0,  0,  0,  0,  1,
          1, 18, 20,  0, 20, 18, 20,  6, 20, 20,  3, 20, 20, 20,  1, 20,  1, 19,
         20, 20,  1,  0,  1,  0, 20,  5, 20,  0,  1,  0,  5,  3,  0,  0,  4, 20,
          1, 20,  1,  3,  3, 20,  0,  0, 20, 20,  3,  5,  1,  0,  4,  1,  1,  0,
          1,  1,  0,  1, 20,  1, 18,  1,  5,  1,  1, 20,  1, 20, 20, 20,  4,  1,
          0,  5,  1, 18,  1,  0,  3,  1,  1, 12, 20,  0, 20, 20,  0, 20,  3, 20,
          1,  0,  0, 20,  0, 20, 20,  0,  1, 20,  1,  0,  1,  3,  4, 20,  1, 20,
          1, 20, 20, 20,  4, 18,  0, 18, 14, 20,  1, 18,  5, 20,  0,  0,  1, 20,
          5,  4,  1, 20,  3, 18,  4,  1,  1, 20,  4,  1,  1, 18,  1, 20,  1,  1,
         

In [17]:
print('Elapsed time:', dt.datetime.now() - START_DT)

Elapsed time: 0:01:04.763266


In [18]:
%%time

NUM_BATCHES = len(dataloader)

with trange(NUM_BATCHES) as bar:
    for x, y in dataloader:
        # print(x.shape, y.shape)
        bar.update(1)

  0%|          | 0/321 [00:00<?, ?it/s]

Wall time: 32min 3s


In [19]:
print('Elapsed time:', dt.datetime.now() - START_DT)

Elapsed time: 0:33:08.666929


In [20]:
test_dataset = JDIDataset(dataset_names=['angular'], rebalance=False)

2021-03-30 00:02:26,236 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:16 - List of dataset_names:['angular']
2021-03-30 00:02:26,238 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:21 - Dataset for angular
2021-03-30 00:02:27,163 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:23 - Dataset shape: (6450, 13)
2021-03-30 00:02:27,164 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:25 - cleaning tag_name from dummy/auxiliary words
2021-03-30 00:02:27,167 - INFO - dataset:dataset.py:59 - select all leafs (nodes which are not parents)
2021-03-30 00:02:27,169 - INFO - dataset:dataset.py:61 - Leafs set size: 3250 (nodes which have no children)
2021-03-30 00:02:27,172 - INFO - dataset:dataset.py:64 - count number of references to leafs
2021-03-30 00:02:27,176 - INFO - dataset:dataset.py:66 - Nodes with leafs as children set size: 1750 (nodes which have leafs as children)
2021-03-30 00:02:27,180 - INFO 

  0%|          | 0/6450 [00:00<?, ?it/s]

2021-03-30 00:02:35,834 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:35 - Dataset shape after merging with parents: (6450, 40)
2021-03-30 00:02:35,835 - WARNING - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:40 - Load LABELS from dataset/annotations/angular.txt


Assigning labels:   0%|          | 0/526 [00:00<?, ?it/s]

2021-03-30 00:02:43,098 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:55 - Concatenate datasets
2021-03-30 00:02:43,106 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:57 - Dataset shape after reading: (6450, 42)
2021-03-30 00:02:43,111 - WARNING - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:320 - Loading count vectorizer for column "cv_class": model/count_vectorizer_class.pkl
2021-03-30 00:02:43,152 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:333 - class_sm: (6450, 779)
2021-03-30 00:02:43,153 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:66 - Build OHE column for attribute role
2021-03-30 00:02:43,159 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:66 - Build OHE column for attribute type
2021-03-30 00:02:43,166 - INFO - <ipython-input-9-b61a19b556d5>:<ipython-input-9-b61a19b556d5>:66 - Build OHE column for attribute ui
2021-03-30 00:02:43,172 -

In [21]:
BATCH_SIZE = 1
dataloader = DataLoader(test_dataset, shuffle=False, batch_size = BATCH_SIZE)

In [22]:
%%time

NUM_BATCHES = len(dataloader)

with trange(NUM_BATCHES) as bar:
    for x, y in dataloader:
        #print(x.shape, y)
        bar.update(1)

  0%|          | 0/6450 [00:00<?, ?it/s]

Wall time: 11 s


In [23]:
print('Elapsed time:', dt.datetime.now() - START_DT)

Elapsed time: 0:33:37.087409
